# Extremely Minimal, Dirty Bleak Scanner implementation 

In [1]:
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from transitions.extensions.asyncio import AsyncMachine
import asyncio

In [3]:
from bleak import BleakClient, BleakScanner
from bleak.backends.device import BLEDevice

In [4]:
from pycycling.sterzo import Sterzo

In [5]:
class BleakModel:
    '''
    This class is a transitions.AsyncModel wrapper around the BleakScanner class.
    It is used to scan for Bluetooth Low Energy devices and return the results within the state machine framework.
    '''
    bt_devices = {} # class variable to store the discovered devices, since we can only have one BleakScanner
    stop_scan_event = asyncio.Event() # class variable to stop the scan
    def __init__(self):
        self.bleak_client: BleakClient = None
        self.ble_device: BLEDevice = None
        self.connection_target = None
        self.stop_streaming_event = asyncio.Event()
        self.sterzo: Sterzo = None

    def set_target_address(self, address):
        if address in BleakModel.bt_devices:
            self.connection_target = address
            return True
        else:
            return ValueError(f"Address {address} not found in discovered devices")

    async def bt_scan(self):        
        # Start the scanner with the detection callback
        def detection_callback(device, advertisement_data):
            BleakModel.bt_devices[device.address] = device
        async with BleakScanner(detection_callback) as scanner:
            await self.stop_scan_event.wait() # continues to scan until stop_scan_event is set

    async def stop_bt_scan(self):
        self.stop_scan_event.set()

    async def connect_to_device(self):
        if len(BleakModel.bt_devices) == 0:
            return False
       # ble_device = BleakModel.bt_devices[self.connection_target][0] # it's best to feed bleak.BLEDevice objects to the BleakClient
        self.ble_device = (BleakModel.bt_devices.pop(self.connection_target))# remove the device from the list to avoid connecting to it multiple times
        self.bleak_client = BleakClient(self.ble_device) # we don't use the async context manager because we want to access the client object from the disconnect function
        try:
            connected = await self.bleak_client.connect()
            if connected:
                print(f"Connected to {self.connection_target}")
                # Perform operations here, for example, read or write characteristics
                self.sterzo = Sterzo(self.bleak_client)
                return True
            else:
                print(f"Failed to connect to {self.connection_target}")
        except Exception as e:
            print(f"An error occurred: {e}")
            return False
        return True
    
    async def disconnect_from_device(self):
        BleakModel.bt_devices[self.connection_target] = self.ble_device # put it back in the list
        await self.bleak_client.disconnect()
        print(f"Disconnected from {self.bleak_client.address}")

    async def stream_from_device(self):
        self.stop_streaming_event.clear()
        def steering_handler(steering_angle):
            print(steering_angle)
        
        self.sterzo.set_steering_measurement_callback(steering_handler)
        await self.sterzo.enable_steering_measurement_notifications()
        await self.stop_streaming_event.wait()
        print("Started streaming")
        return True

    async def stop_stream_from_device(self):
        self.stop_streaming_event.set()
        try:
            await self.sterzo.disable_steering_measurement_notifications()
            print("Stopped streaming")
            return True
        except Exception as e:
            print(f"An error occurred while stopping streaming: {e}")
            return False

    

In [6]:

model = BleakModel()

transitions = []

machine = AsyncMachine(model, states=["Init", "Scanning", "Connected", "Streaming"], transitions=transitions, initial='Init')

machine.add_transition(
    trigger="start_scan",
    source="Init",
    dest="Scanning",
    after="bt_scan"
)

machine.add_transition(
    trigger="stop_scan",
    source="Scanning",
    dest="Init",
    after="stop_bt_scan"
)

machine.add_transition(
    trigger="connect",
    source="Init",
    dest="Connected",
    conditions="connect_to_device",
)

machine.add_transition(
    trigger="disconnect",
    source="Connected",
    dest="Init",
    before="disconnect_from_device"
    
)

machine.add_transition(
    trigger="stream",
    source="Connected",
    dest="Streaming",
    after="stream_from_device"
)

machine.add_transition(
    trigger="stop_stream",
    source="Streaming",
    dest="Connected",
    before="stop_stream_from_device"
)

In [7]:
await model.connect() # should Fail, because we're trying to connect to a device without scanning first
# specifically, `is_device_list_empty` will return True, and the `unless` condition will fail

False

In [8]:
# Since we're in a Jupyter notebook, 
# we need to use `create_task()` instead of `get_event_loop().run_until_complete()`
# to use the existing event loop which Jupyter runs on.
asyncio.create_task(model.start_scan())
# if we used
# `await model.start_scan()`
# then it would work, but it would block until the scan is finished, which is not what we want,
# because below we will show how the `model.bt_devices` dictionary gets filled with the results of the scan.

<Task pending name='Task-7' coro=<AsyncEvent.trigger() running at /Users/tensorturtle/Library/Python/3.9/lib/python/site-packages/transitions/extensions/asyncio.py:166>>

In [9]:
async def stop_after_delay(delay, model):
    print("Starting sleep for 2 seconds (giving the scanner time to scan)")
    await asyncio.sleep(delay)
    print("Slept for 2 seconds (sending end signal to scanner)")
    await model.stop_scan()
    print("Stopped scanning")
    

asyncio.create_task(stop_after_delay(2, model))

<Task pending name='Task-9' coro=<stop_after_delay() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_75754/921128545.py:1>>

In [10]:
async def print_as_we_go():
    for i in range(20):
        print(f"{len(BleakModel.bt_devices)} devices found in {round(i*0.05,2)} seconds.")
        await asyncio.sleep(0.05)

await print_as_we_go()

Starting sleep for 2 seconds (giving the scanner time to scan)
0 devices found in 0.0 seconds.
1 devices found in 0.05 seconds.
4 devices found in 0.1 seconds.
6 devices found in 0.15 seconds.
6 devices found in 0.2 seconds.
7 devices found in 0.25 seconds.
7 devices found in 0.3 seconds.
8 devices found in 0.35 seconds.
8 devices found in 0.4 seconds.
9 devices found in 0.45 seconds.
9 devices found in 0.5 seconds.
9 devices found in 0.55 seconds.
9 devices found in 0.6 seconds.
9 devices found in 0.65 seconds.
9 devices found in 0.7 seconds.
9 devices found in 0.75 seconds.
9 devices found in 0.8 seconds.
9 devices found in 0.85 seconds.
9 devices found in 0.9 seconds.
9 devices found in 0.95 seconds.


In [11]:
model.bt_devices

{'14863C4D-BF71-4EA3-C6B4-98001056AAF8': BLEDevice(14863C4D-BF71-4EA3-C6B4-98001056AAF8, WHOOPDEDOO),
 '22ACD010-1CE0-A34E-795A-30CE2A26446B': BLEDevice(22ACD010-1CE0-A34E-795A-30CE2A26446B, None),
 '84FBE956-5FC9-C70B-109E-D86D66488FFD': BLEDevice(84FBE956-5FC9-C70B-109E-D86D66488FFD, None),
 '9E41F52B-BDEF-3C85-F51D-E0DE23B7CB41': BLEDevice(9E41F52B-BDEF-3C85-F51D-E0DE23B7CB41, Study),
 '3FEC3389-7F84-AFC4-6EF1-BDE6CD37F862': BLEDevice(3FEC3389-7F84-AFC4-6EF1-BDE6CD37F862, Green Apple),
 '37DA5EA4-DB46-2D6D-9E8F-A7B7D4DBCCA4': BLEDevice(37DA5EA4-DB46-2D6D-9E8F-A7B7D4DBCCA4, GiGA Genie 3_F70E),
 'DC2343A7-C46E-E1BB-30E9-BC6E274B4A39': BLEDevice(DC2343A7-C46E-E1BB-30E9-BC6E274B4A39, Fast Fred),
 'CB9E513B-88D8-5AE7-F447-3A332B6252E8': BLEDevice(CB9E513B-88D8-5AE7-F447-3A332B6252E8, None),
 'A3133322-B7D2-25F0-E893-71E7D984F383': BLEDevice(A3133322-B7D2-25F0-E893-71E7D984F383, None)}

In [12]:
model.set_target_address("715D8603-DC4C-2994-C0CD-2BC5A93E0B38")

ValueError('Address 715D8603-DC4C-2994-C0CD-2BC5A93E0B38 not found in discovered devices')

In [13]:
await asyncio.sleep(3)

Slept for 2 seconds (sending end signal to scanner)
Stopped scanning


In [14]:
await model.connect()

KeyError: None

In [ ]:
model.state

'Connected'

In [ ]:
model.connection_target

'715D8603-DC4C-2994-C0CD-2BC5A93E0B38'

In [ ]:
# a second bluetooth model

model2 = BleakModel()
machine.add_model(model2)

In [ ]:
async def run_stream():
    await model.stream()

asyncio.create_task(run_stream())

<Task pending name='Task-15' coro=<run_stream() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_49442/3697076873.py:1>>

In [ ]:
await asyncio.sleep(10)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1.393035888671875
1.4605560302734375
1.8665771484375
1.97076416015625
1.393035888671875
1.6665573120117188
1.8665771484375
1.6969375610351562
1.8378829956054688
1.3251113891601562
2.0758132934570312
1.97076416015625
1.25677490234375
1.76324462890625
1.2912826538085938
1.6665573120117188
1.5631332397460938
1.7336273193359375
1.5631332397460938
1.7708511352539062
1.97076416015625
1.4956283569335938
1.7336273193359375
1.8003005981445312
1.6665573120117188
1.5631332397460938
1.6665573120117188
1.8003005981445312
1.5990829467773438
1.8665771484375
1.8003005981445312
1.6665573120117188
1.6665573120117188
1.97076416015625
1.393035888671875
1.9045257568359375
1.6969375610351562
1.8003005981445312
1.5631332397460938
1.393035888671875
1.97076416015625
2.0758132934570312
1.6665573120117188
1.8003005981445312
2.0758132934570312
1.1549911499023438
1.76324462890

In [ ]:
await model.stop_stream()

1.932464599609375
Stopped streaming


True

In [ ]:
await model.disconnect()

Disconnected from 715D8603-DC4C-2994-C0CD-2BC5A93E0B38


True

# Important Todo

After stream is stopped, we can't simpy re-start it.

We need to re-create a BleakClient object, which means we need to retreat back to Init state
in order to call `connect()`, which re-creates that BleakClient object.

In our FSM, we need to remove the path from Connected -> Streaming because that's not possible.

In [ ]:
model.state

'Init'

In [ ]:
await model.connect()

Connected to 715D8603-DC4C-2994-C0CD-2BC5A93E0B38


True

In [ ]:
async def run_stream():
    await model.stream()

asyncio.create_task(run_stream())

<Task pending name='Task-24' coro=<run_stream() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_49442/3697076873.py:1>>

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1.6302337646484375
1.8665771484375
1.4956283569335938
1.5276870727539062
1.4605560302734375
1.7336273193359375
1.8003005981445312
1.932464599609375
1.82916259765625
1.393035888671875
1.8378829956054688
2.0758132934570312
1.1549911499023438
1.8378829956054688
1.8003005981445312
1.4277267456054688
1.5631332397460938
1.8003005981445312
1.7336273193359375
1.5631332397460938
1.5631332397460938
1.393035888671875
1.6665573120117188
1.7336273193359375
1.393035888671875
1.6969375610351562
1.0170669555664062
1.8665771484375
1.393035888671875
1.76324462890625
1.9045257568359375
1.5631332397460938
1.6665573120117188
2.1020660400390625
1.2233428955078125
1.8665771484375
1.76324462890625
1.4956283569335938
2.0758132934570312
1.5990829467773438
1.8378829956054688
1.8003005981445312
1.6302337646484375
1.393035888671875
1.7336273193359375
2.009613037109375
1.7632446289